<a href="https://colab.research.google.com/github/YarkoMarko/DecisionTree/blob/main/module3/%D0%94%D0%97_8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [64]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [65]:
df = pd.read_csv("https://raw.githubusercontent.com/HalyshAnton/IT-Step-Pyton-AI/main/module2/data/ship_passengers.csv",
                 index_col="Unnamed: 0")

In [66]:
df.head()

,pclass,sex,age,sibsp,parch,fare,embarked
0,3,male,22.0,1,0,7.2500,S
1,1,female,38.0,1,0,71.2833,C
2,3,female,26.0,0,0,7.9250,S
3,1,female,35.0,1,0,53.1000,S
4,3,male,35.0,0,0,8.0500,S


In [67]:
df = df[['pclass', 'age', 'embarked', 'fare']]
df.head()

,pclass,age,embarked,fare
0,3,22.0,S,7.2500
1,1,38.0,C,71.2833
2,3,26.0,S,7.9250
3,1,35.0,S,53.1000
4,3,35.0,S,8.0500


# Завдання 1
Очистіть дані від викидів(лише `fare`), розділіть на тренувальну та тестову чатини

In [68]:
df.shape

(891, 4)

In [69]:
def remove(df, column):
  column_ = df[column]

  q1 = column_.quantile(q=0.25)
  q3 = column_.quantile(q=0.75)
  iqr = q3 - q1

  lower = q1 - iqr * 1.5
  upper = q3 + iqr * 1.5

  mask = (column_ >= lower) & (column_ <= upper)

  return df[mask]

df = remove(df, "fare")

In [70]:
df.shape

(775, 4)

In [71]:
y = df["fare"]
X = df.drop(columns=["fare"])

In [72]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.8, random_state=42)

# Завдання 2
Створіть Pipeline для обробки даних

In [73]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

In [74]:
num_columns = X.select_dtypes(include="number").columns
cat_columns = X.select_dtypes(include="object").columns

In [75]:
num_transformer = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="median"))
    ]
)

cat_transformer = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("encoder", OneHotEncoder(sparse_output=False))
    ]
)

preprocessor = ColumnTransformer(
    transformers=[
        ("num", num_transformer, num_columns),
        ("cat", cat_transformer, cat_columns)
    ]
)

preprocessor.set_output(transform="pandas")

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='median'))]),
                                 Index(['pclass', 'age'], dtype='object')),
                                ('cat',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('encoder',
                                                  OneHotEncoder(sparse_output=False))]),
                                 Index(['embarked'], dtype='object'))])

In [55]:
num_transformer = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="mean"))
    ]
)

cat_transformer = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("encoder", OneHotEncoder(sparse_output=False))
    ]
)

preprocessor = ColumnTransformer(
    transformers=[
        ("num", num_transformer, num_columns),
        ("cat", cat_transformer, cat_columns)
    ]
)

preprocessor.set_output(transform="pandas")

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('imputer', SimpleImputer())]),
                                 Index(['pclass', 'age'], dtype='object')),
                                ('cat',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('encoder',
                                                  OneHotEncoder(sparse_output=False))]),
                                 Index(['embarked'], dtype='object'))])

In [76]:
preprocessor.fit(X_train)

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='median'))]),
                                 Index(['pclass', 'age'], dtype='object')),
                                ('cat',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('encoder',
                                                  OneHotEncoder(sparse_output=False))]),
                                 Index(['embarked'], dtype='object'))])

# Завдання 3
Створіть остаточну модель та натренуйте її

In [77]:
from sklearn.tree import DecisionTreeRegressor

In [58]:
model = Pipeline(
    steps=[
        ("prop", preprocessor),
        ("model", DecisionTreeRegressor(max_depth=4,
                                        random_state=42))
    ]
)

In [78]:
model = Pipeline(
    steps=[
        ("prop", preprocessor),
        ("model", DecisionTreeRegressor(max_depth=6,
                                        random_state=42))
    ]
)

In [79]:
model.fit(X_train, y_train)

Pipeline(steps=[('prop',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median'))]),
                                                  Index(['pclass', 'age'], dtype='object')),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encoder',
                                                                   OneHotEncoder(sparse_output=False))]),
                                                  Index(['embarked'], dtype='object'))])),
                ('model', DecisionTreeRegressor(max_depth=6, random_state=42))])

# Завдання 4
Виведіть основні метрики моделі

In [80]:
from sklearn import metrics

y_pred = model.predict(X_test)

def get_metrics(y_true, y_pred, name="model"):
  df_ = pd.DataFrame()

  df_.loc["MAE", name] = metrics.mean_absolute_error(y_true, y_pred)
  df_.loc["RMSE", name] = metrics.mean_squared_error(y_true, y_pred) ** 0.5
  df_.loc["R2", name] = metrics.r2_score(y_true, y_pred)

  return df_.round(2)

In [81]:
all_metrics = pd.DataFrame()

In [82]:
all_metrics["train"] = get_metrics(y_train, model.predict(X_train))
all_metrics["test"] = get_metrics(y_test, y_pred)

In [24]:
all_metrics

,train,test
MAE,5.83,7.74
RMSE,8.72,11.07
R2,0.39,0.38


# Завдання 5
Змініть параметри моделі та попробуйте покращити результат

Попробуйте:
* не видаляти викиди
* змінити обробку даних(imputer та кодування категоріальних даних)
* змінити параметри дерева(глибина, кількість точок у листках тощо

In [43]:
all_metrics

,train,test
MAE,11.61,18.33
RMSE,21.35,43.67
R2,0.67,0.30


In [63]:
all_metrics

,train,test
MAE,5.75,7.74
RMSE,8.67,11.10
R2,0.39,0.38


In [83]:
all_metrics

,train,test
MAE,4.82,7.91
RMSE,8.00,11.69
R2,0.49,0.31


# Завдання 6
Збережіть модель

In [84]:
import joblib

joblib.dump(model, "model.pkl")

['model.pkl']